# Tensorflow Softmax

## Role of placeholder

The placeholder allows you to feed data into the computation graph by way of feed dictionaries. This gives you an entry point from your non-TensorFlow code in Python so you can run a graph on new inputs.

## Automatic Differentiation

TensorFlow's automatic differentiation obviates the need for explicit gradient definition because it decomposes the graph into distinct `Op` components, each of which it has defined the gradient for. It then applies the chain rule to the graph, computes all relevant gradients, and runs a special training `Op` apply updates to the affected `Tensor` components.

# Deep NER

Named Entity Recognition involves predicting one of 4 classes (Person, Organization, Location, Miscellaneous) or a 5th null class (not a named entity) - here we are modeling it using a neural network.

## Gradients

### Base Model

The model is a 1-hidden layer neural net that classifies whether the center word is a certain class based on a small window. The windows is represented as a concatenation of 1-hot vectors that share an embedding lookup and pass through a hidden layer and softmax output layer.

$x^{(t)} = [x_{t-1} L, x_t L, x_{t+1} L] \in \Real^{3d}$

$L \in R^{\|V| \times d}$ is an embedding matrix

$h = \tanh(x^{(t)} W + b_1)$

$\hat y = softmax(h U + b_2)$

$J(\theta) = CE(y, \hat y) = - \sum_{i = 1}^5 y_i \log \hat y_i$

The cost is the sum cross entropy cost over all 4 classes plus the 5th null class, which turns out to have the most examples.